In [1]:
library(tidyverse)
library(openxlsx)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
hubgenes = qs::qread('_targets/objects/hubgenes_Agrp')
hubgenes %>% head

gene,name,value,label
<chr>,<chr>,<dbl>,<chr>
St18,kME4,0.7473009,Agrp
Sulf1,kME4,0.6492369,Agrp
Kcnb2,kME2,0.6038895,Agrp
Terf1,kME2,0.5838000,Agrp
Crispld1,kME10,0.7535446,Agrp
B3gat2,kME9,0.7667428,Agrp


In [3]:
make_hubgenes_summary = function(hubgenes){
    hubgenes_summary = hubgenes %>% 
    mutate(new_name = name) %>%
    mutate(new_name = stringr::str_replace(new_name, 'kME', '')) %>% 
    mutate(new_name = stringr::str_pad(new_name, 2, 'left', pad='0')) %>%
    mutate(new_name = paste0(label, '_M', new_name)) %>%
    group_by(new_name) %>%
    arrange(desc(value)) %>%
    select(gene, new_name) %>%
    distinct %>%
    mutate(genes = paste0(gene, collapse=', ')) %>%
    mutate(module_size = n()) %>%
    select(new_name, module_size, genes) %>%
    distinct %>%
    arrange(new_name) %>%
    rowwise %>% 
    mutate(label = {stringr::str_split(new_name, fixed('_')) %>%
                    unlist %>%
                    `[[`(1)}) %>%
    rename(module_name = new_name) %>%
    relocate(label) %>%
    ungroup
    
    hubgenes_summary
}


In [4]:
hubgenes_summary = make_hubgenes_summary(hubgenes)
hubgenes_summary


label,module_name,module_size,genes
<chr>,<chr>,<int>,<chr>
Agrp,Agrp_M01,387,"Cpe, Lepr, N4bp2l1, Fndc3a, Ncoa2, Syndig1, Ell2, Marchf11, ENSMUSG00000092090, Atxn10, Rad54l2, Dmxl1, Zwint, Pde10a, Phf8, Fam126b, Asap1, Herc2, Slc35f3, Sv2c, Oxr1, Prkca, Pak3, Uhrf2, Prkacb, Sf3b1, Ap2b1, Vcl, Ankrd17, Ptprn, Gpr158, Wdfy1, Slc30a4, Rab3b, Chd7, Spred2, Bbx, Lncpint, ENSMUSG00000118257, Cpeb4, Spred1, Trim9, Sik3, Myof, Etv5, Hs3st1, Pam, Clvs1, Homer1, Sipa1l2, Psme4, Hacd3, Fryl, Ift57, Susd6, ENSMUSG00000109006, Sbf2, Cop1, Cds1, Xkr6, Ttc39c, Ubr3, Tpp2, Foxo1, Adcy2, Slc6a17, Nup98, Aff4, Ranbp2, Malat1, B4galt1, Gm5820, Braf, Vat1l, Ralgapa1, Ubr4, Ghr, N4bp2l2, Stk38l, Zfp697, Usp29, Chic1, Dusp16, Cltc, Fgf1, Hsd17b12, ENSMUSG00000108854, Npy, Actr3b, Samd4, Ntrk2, Ddx50, Atp11a, Garre1, Trpc1, Ahi1, Brd4, Fam193a, Cttnbp2nl, Inpp5a, Unc13a, Sorcs1, Mapk10, Srrm4, Plce1, Ap1g1, Hivep1, Rbfox2, Pde8a, ENSMUSG00000075020, Tut4, Zc3h12c, Rab3gap2, Cers6, Snap25, Ndfip2, Gkap1, Pcsk1, Pkd1, Dclk1, Fosl2, Fyb2, Spry2, Noct, Srxn1, Ntng1, Jak1, Cdkl5, Thoc2l, Nr4a1, Klf9, Rorb, Cdkn1a, Orc4, Ppm1b, Tet2, Klf12, Clip1, Stac2, Osbp2, Ap2a2, Dgkd, Glg1, Pik3r3, Rasgrf1, Btg3, Atp6v1b2, Hivep2, Med14, ENSMUSG00000085609, Usp34, Arih2, Phf20l1, Sik2, Osbpl1a, Mast2, Tomm70a, Tnrc6a, Fam107b, Trak2, Peg3, Atp1b3, Fras1, Nexmif, Spred3, Zfp948, Creb3l2, Rhoj, Ndel1, Adra1b, Garem1, Edem2, Mark1, Nktr, Pvr, Rel, ENSMUSG00000079065, Fbxw11, Dock10, Kras, Slc41a2, Bard1, Usp15, Arid4b, P3h2, Chka, Ywhaz, Clstn3, Acsl4, Nf1, Zfp426, Map7d2, Igsf11, Trank1, Rasgrf2, Unk, Kdm2a, Usp9x, Bach2, Stk40, Mprip, Cystm1, Tfrc, Abca5, Dhx15, Sh3rf1, Ppm1e, Vgf, Marchf4, Nhlrc2, Taf1, Map2k3, Btrc, Nab1, Pja2, Arih1, Septin7, Shisal2a, Smarca5, Plekha5, Snx3, Trip12, Btaf1, Pdzd8, Nfx1, Gfra1, Cux1, Ube2k, Dtnb, Dpm1, Uvrag, Eprs, Prmt2, Gmeb2, Rap1b, Cd47, Pex5, Htt, Spry4, Gbf1, Cab39, Pip5k1a, Ric8b, Mast4, Kpna1, Npas4, Il1r1, Slc7a6, Usp13, Mon2, ENSMUSG00000112470, Sec14l1, Ppef1, Arf4, Ppp2r2d, Bcl2l13, Crem, Smchd1, Grm5, Uhrf1bp1l, Slc8a1, Rbm39, Ndfip1, Ubl3, Atp2a2, Dnajc1, Adamts6, Rin2, Usp37, Gad1, Brca2, Psmd14, Rbms2, Grip1, ENSMUSG00000109814, Rab6a, Nfil3, Pcgf5, Rasal2, Atp13a3, Atf6, Rbm26, Adrb2, Prkar1a, Tent5a, Nsd3, Ext1, Zfp46, Psmd1, Ythdc1, Cnot6l, Gng4, Nfkbiz, Adgra1, Bdp1, Cul3, Cdk13, Hunk, Ppm1l, Ubap2l, Scg2, Alkal2, E2f3, Ccnl1, Kif2a, Slc25a3, Ttc19, Rad18, Dst, Slc25a25, Nckap1, Cflar, Mtdh, Hhat, Ppm1d, Myo9b, Pspc1, Elovl5, Ankrd11, Rbpj, Tulp4, Rasa2, Usp32, Sf3b3, Pfkp, Hectd1, Lurap1l, Hprt, Map2, Mfn1, Atp9b, Ecpas, Arhgap44, Akap11, Mtmr1, Nin, Csnk1a1, Ret, Uty, Map3k2, 5330434G04Rik, Ago3, Abhd2, Nmnat2, ENSMUSG00000097451, Per1, Ptprk, Tex10, Jarid2, Txnl1, Mdn1, Dennd5b, Rab3c, Rock2, Mfap3l, Tra2b, Atf2, Gpr137c, Farp1, Pafah1b1, Azin1, Rapgef2, Apbb2, Cpsf6, Tsc22d2, Nr3c1, Dlk1, Tle4, Ammecr1, Chm, Top1, Prepl, Ambra1, Hdac5, Tcf25, Ascc3, Thoc2, Nrd1, Csde1, Rtn4, Fmn2, Hnrnpu, Zfhx2, Lims1, Cmip, Inpp5f, Kif21a"
Agrp,Agrp_M02,246,"Add3, Shank2, Maml2, Ksr2, Trerf1, Pak5, Srgap3, Dcdc2a, Mtus2, Phactr2, Camk2a, Dab1, Vwa5b1, Fam13a, Ssbp2, Cxxc4, Cacnb4, Cfap44, L3mbtl1, St8sia1, Kcnh8, D130043K22Rik, Nsg2, Spock3, Lcorl, Asph, Kcnn3, Epb41l3, Hlf, Dpysl3, Ccdc85a, Gria1, Smoc1, Man2a1, Lypd6, Ankrd6, Gabra3, Ttc28, Dnah9, Prkn, Tet1, Anxa6, Ypel2, Mmp16, Ulk4, Katnal2, Manba, Bbs9, Chd9, Kndc1, Pag1, Dzip1, Eml1, Nrg2, D3Ertd751e, Ksr1, Tafa5, Mcf2l, Fndc1, Gm12296, Gm1604b, Tef, Drc1, Magi1, Unc79, Tjp1, ENSMUSG00000097452, Arsb, Amph, Naaladl2, Prtg, Abcc4, D630045J12Rik, Cdkl1, Rph3al, P4ha3, ENSMUSG00000116760, Ccdc187, Plekha6, Scml4, Nxph1, Mir100hg, Ppp3ca, Syt17, Erich3, Armc2, Crocc, AW551984, Adcy5, Stmn2, Fn3k, Dpysl5, Cgn, Esyt3, Cdin1, B3galt5, Hrh3, Dapk1, Angpt1, Pou2f2, Ccdc30, Plec, Cngb1, Ncoa7, Rnf144a, Cpne8, Stk32a, Mir99ahg, Hspa12a, Palm, Trim2, Osbpl9, Susd1, Nalcn, Mast1, Radil, Scn1a, Gm1604a, Slain1, Lrrc3b, Lama1, Fig4, Rere, Dpysl2, Shld2, Syne2, Car8, Bmp3, Slit3, C

In [5]:
top_table = qs::qread('_targets/objects/top_table_Agrp')

make_hubgenes_tt_summary = function(hubgenes, top_table){
    hubgenes_summary = make_hubgenes_summary(hubgenes)
    
    hubgenes_tt_summary = top_table %>%
        rename(module_name = module) %>%
        select(module_name, comparison, logFC, adj.P.Val, significant) %>%
        left_join(hubgenes_summary, ., by = 'module_name') %>%
        relocate(genes, .after = significant) %>%
        arrange(label, module_name, comparison)
    
    hubgenes_tt_summary    
}


In [6]:
chst = qs::qread('_targets/objects/combined_hubgenes_summary_tt_all')
chst %>% head(3)

label,module_name,module_size,comparison,logFC,adj.P.Val,significant,genes
<chr>,<chr>,<int>,<chr>,<dbl>,<dbl>,<lgl>,<chr>
a1_Tany__Astrocytes,a1_Tany__Astrocytes_M01,518,Day14.obob.FGF1vsVehPF,3.823056e-03,0.063967972,FALSE,"Rundc3b, Dscam, Kcnq5, Marchf1, Edil3, Rexo5, Mtss1, Mirg, Gfod1, Cers6, Dtx2, Wdr45b, Zbtb2, ENSMUSG00000097311, Rnf40, Ube2i, Camk2b, Snrpb, Nav1, Sntg1, Aplp2, Tmsb4x, Atp1a1, Ipw, Rab3c, Ncoa3, Ndufa11, Btg3, Smdt1, Xpc, ENSMUSG00000068151, Pde3b, Usp12, Cnot6l, Samhd1, Phldb1, Snap25, Lmo7, Nckap5l, Ahctf1, Rps19, Zfp804b, Pgrmc2, Tle3, Slc25a25, Cct7, Txndc15, Rps8, Elmo1, Frmpd4, Kif5c, Lrfn5, Ncl, Ccnyl1, Klhl23, Rps14, Atp2b1, Serinc3, Rab1a, Rpl10a, Ywhaq, Prkg1, Sergef, Usp42, Pcsk2, Scn3a, Sap130, Cic, Cdc37, Slc31a1, Plbd2, Pde7a, Sod2, Madd, Atp5d, Ppfia2, Frmd5, ENSMUSG00000108532, Psma7, Nfe2l2, Sqle, Cntrl, Dpp10, Dnajb14, Fbxl5, Cops4, Cd200, Mapk8ip3, Exd2, Cd9, Gclc, Ippk, Slc7a6, Psma2, Mrps5, Cnih1, Hectd2, Eif3k, Rasgrf2, Ska2, Brk1, Me2, Serpini1, Tmem181a, Tyw1, Wdr1, Ttc38, Prkar1b, Rpl18, Ubb, Arl8b, Eef1a1, Rps9, Elavl4, Arhgap31, Ralgapb, Tnrc6c, Dmxl2, App, Elavl3, Pcbp3, Dcaf7, Esd, Zdhhc2, Golm2, Cap1, Ust, Spg21, Gtpbp8, Ehhadh, Josd1, Sav1, Arhgap39, Cntn6, Myef2, Hectd4, Rpl3, Ano4, Cul4a, Atp5j, Lrrfip1, Sgms1, Isca2, Ndrg4, Bicdl1, Btbd8, Dync1i2, Nol8, Nsg2, Kpna3, Dipk1a, ENSMUSG00000039530, Eif4g2, Tubb2a, Cdh18, Arcn1, Dtnbp1, Lactb, Tmeff1, Ebpl, Farsa, Decr2, Pfdn5, Fra10ac1, Fxyd6, Olfm1, Tmem19, Cetn3, Tenm1, Gtf2a1, Cops2, Sec63, Lingo2, Ptma, Rpl13, Psmb3, Hcfc2, Rab33b, Ppme1, Nova2, Grm5, Aqr, Arf4, Ero1a, Peg10, Proser1, Acat1, Ugcg, Rnf138, Ncoa7, Psmc3, Paip2, Pafah1b2, Hecw1, Usp22, Fam20b, Slain1, Prpf38a, Phc2, Fstl5, Phyh, Pdia3, Lmbrd2, Reck, Sptlc2, Lemd3, Ccdc186, Cpne6, Rab21, Nexmif, Akt2, Kcnab1, Mif, Osgep, Gabra3, Rbm4b, Vsnl1, Atraid, Slc30a5, Map1b, Ggnbp2, Fam120a, Rnmt, Sf3b6, Ptdss2, Cox8a, Rrp1b, Znrf2, Supt5, Fxr2, Morf4l1, Spock3, Dgcr8, Slc24a2, Faim2, Srsf3, Lima1, Fez2, Zkscan1, ENSMUSG00000114304, Btg1, Zfp326, Tbc1d9, Bri3bp, Anapc5, Ccdc47, Psen1, Polr2b, Zmat2, Dctn6, Lrrk1, Grk2, Slc44a2, Puf60, Cwc15, Whrn, Tbca, Dpy19l4, Kctd9, Caprin1, Psma6, Zdbf2, Rnf114, Pgap1, Cacna1b, Coq10b, Ndufs2, Pink1, Brinp1, Med13, Fam169a, Prdm11, Zfp655, Casd1, Ndst2, Arfgap3, Rps20, Nipa2, Etaa1, Nucb2, Rit1, Strn, Ppp4r2, Zfp758, Mllt1, Ap1b1, Dtd1, Rbm27, Cpt1c, Wrnip1, Galnt13, 5730455P16Rik, Lyrm4, Pdcd6ip, Stx7, Dgcr2, Fth1, Inpp5a, Pwwp2a, Leo1, Ankrd13c, Ppib, Sbds, Pdzd4, Lrrn1, Ndufa5, Gm32856, Rwdd1, Mesd, Rpl21, Fbxo10, Lars2, Ccdc12, Ap2b1, Nudcd3, Fam118b, C87436, Ndufb9, Rnf14, Adgrg1, Atg4a, Cntn4, Kbtbd2, Gramd1a, Eif3h, Xkr4, Galnt17, Coq10a, Ppp2r1a, Chga, Stim1, Sec31a, Phtf1, Bcl2l1, Snx24, Ube2cbp, Setdb2, Nras, Ube2e3, Gt(ROSA)26Sor, Fah, Pfdn2, Cntn3, Wdcp, Akr1e1, Brox, Fkbp14, Slc25a37, Ndufb10, Xpo4, Denn2b, P4hb, Ccdc85c, Bsg, Grb10, Smim13, Ccdc77, Fyttd1, Rnf32, Dlc1, Phaf1, Ap3d1, Sbno2, Rictor, Gm10373, Gstm1, Sp3, Exoc2, Srsf4, Cr1l, Trio, Chid1, Anp32b, Arpc2, Elovl5, Sgcz, Ncdn, Dynll1, Hcn1, Bex2, Get4, Vstm2a, Eral1, Atl3, Brd3, Lipo3, Wasf1, Srp9, 2810013P06Rik, Sdhaf3, Elavl2, Snx16, Far1, Exosc1, Pip5k1c, Eno2, Cul4b, Sesn1, Grcc10, Ino80, Tmf1, Samm50, St3gal3, Cpsf2, Gpr89, Lrpap1, Kat8, Alg13, Niban2, Cdh6, Polb, Gk5, Rhoa, Fastkd1, Rasgef1a, Pithd1, Mrpl23, Chfr, Degs1, Heatr5a, Satb1, 2610044O15Rik8, Dtx4, Eif1ax, Alg2, Pomt2, Lrrc58, Dnajc8, B4galt6, Zmiz1, Nek3, Hmgn1, Tsc2, Stat2, Malsu1, Lrp5, L3mbtl3, Tmem30a, Use1, Rpl23, Zfp874a, Bsn, Hspa13, Dab2ip, Akap17b, Samd12, Map2k5, Hspa8, 2300009A05Rik, Arrdc3, Disp2, Zfp595, Atrn, Ccdc28a, Tsn, Nop58, Il1rapl2, Pgam1, Epb41l3, Ppp1r36, ENSMUSG00000092368, Snx3, Atp6v1b2, Naa50, Rpl8, Tbc1d22a, Kif13b, Tac1, Slc25a3, Med23, Mettl26, Map2k2, Hus1, Alkbh3, Nadk2, Trpm7, Shoc2, Cox6c, Fbxo30, Zfp62, Zfp287, Kif3b, Ndufaf7, Pnpt1, Lmf1, Wdr25, Mgrn1, Vrk1, Zfp110, Entpd6, Ubxn4, 1110059E24Rik, Eif4a1, Usp40, Blzf1, Bmper, Epc1, Immp1l, Tsnax, Ppp1r11, S

In [7]:
chst %>% distinct(comparison) %>% pull(comparison) %>% dput

c("Day14.obob.FGF1vsVehPF", "Day5.obob.FGF1vsVehPF", "Day5vsDay14.BL6.VehPF", 
"Day5vsDay14.obob.VehPF")


In [8]:
chst = qs::qread('_targets/objects/combined_hubgenes_summary_tt_all')

comparison_levels = c("Day5.obob.FGF1vsVehPF",
                      "Day14.obob.FGF1vsVehPF",
                      "Day5vsDay14.obob.VehPF",
                      "Day5vsDay14.BL6.VehPF")

chst %>% 
filter(!is.na(comparison)) %>% 
# mutate(comparison = factor(comparison, levels = comparison_levels)) %>%
select(label, module_name, module_size, comparison, significant) %>% 
pivot_wider(names_from = comparison, values_from = significant) %>%
relocate(any_of(comparison_levels), .after=module_size) %>%
mutate(FGF1_Day5 = case_when((Day5.obob.FGF1vsVehPF == TRUE) & !(Day5vsDay14.obob.VehPF == TRUE | Day5vsDay14.BL6.VehPF == TRUE) ~ TRUE,
                             TRUE ~ FALSE)) %>%
mutate(FGF1_Day14 = case_when((Day14.obob.FGF1vsVehPF == TRUE) & !(Day5vsDay14.obob.VehPF == TRUE | Day5vsDay14.BL6.VehPF == TRUE) ~ TRUE,
                              TRUE ~ FALSE)) %>%
mutate(FGF1_both_days = case_when(((FGF1_Day5 == TRUE) & (FGF1_Day14 == TRUE)) ~ TRUE,
                                   TRUE ~ FALSE)) %>%
relocate(any_of(c("FGF1_Day5", "FGF1_Day14", "FGF1_both_days")), .after=module_size)

label,module_name,module_size,FGF1_Day5,FGF1_Day14,FGF1_both_days,Day5.obob.FGF1vsVehPF,Day14.obob.FGF1vsVehPF,Day5vsDay14.obob.VehPF,Day5vsDay14.BL6.VehPF
<chr>,<chr>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
a1_Tany__Astrocytes,a1_Tany__Astrocytes_M01,518,TRUE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE
a1_Tany__Astrocytes,a1_Tany__Astrocytes_M02,317,TRUE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE
a1_Tany__Astrocytes,a1_Tany__Astrocytes_M03,306,TRUE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE
a1_Tany__Astrocytes,a1_Tany__Astrocytes_M04,282,TRUE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE
a1_Tany__Astrocytes,a1_Tany__Astrocytes_M05,211,TRUE,TRUE,TRUE,TRUE,TRUE,FALSE,FALSE
a1_Tany__Astrocytes,a1_Tany__Astrocytes_M06,148,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
a1_Tany__Astrocytes,a1_Tany__Astrocytes_M07,128,TRUE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE
a1_Tany__Astrocytes,a1_Tany__Astrocytes_M08,95,FALSE,FALSE,FALSE,TRUE,FALSE,TRUE,FALSE
a1_Tany__Astrocytes,a1_Tany__Astrocytes_M09,89,TRUE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE


In [9]:
str_replace("Sst_Unc13c__Agrp_M18", "_M\\d{2}", '')

[1] "Sst_Unc13c__Agrp"

In [10]:
make_combined_hubgenes_summary_tt_all_summary = function(combined_hubgenes_summary_tt_all){
    comparison_levels = c("Day5.obob.FGF1vsVehPF",
                          "Day14.obob.FGF1vsVehPF",
                          "Day5vsDay14.obob.VehPF",
                          "Day5vsDay14.BL6.VehPF")

    combined_hubgenes_summary_tt_all %>% 
    filter(!is.na(comparison)) %>% 
    mutate(adj_logFC = case_when(significant == FALSE ~ 0,
                             TRUE ~ logFC)) %>%
    mutate(direction = case_when(adj_logFC == 0 ~ '-',
                                 adj_logFC > 0 ~ 'UP',
                                 adj_logFC < 0 ~ 'DOWN')) %>%
    select(label, module_name, module_size, comparison, direction) %>% 
    pivot_wider(names_from = comparison, values_from = direction) %>%
    relocate(any_of(comparison_levels), .after=module_size) %>%
    mutate(FGF1_Day5 = case_when((Day5.obob.FGF1vsVehPF != '-') & !(Day5vsDay14.obob.VehPF != '-' | Day5vsDay14.BL6.VehPF != '-') ~ Day5.obob.FGF1vsVehPF,
                                 TRUE ~ '-')) %>%
    mutate(FGF1_Day14 = case_when((Day14.obob.FGF1vsVehPF != '-') & !(Day5vsDay14.obob.VehPF != '-' | Day5vsDay14.BL6.VehPF != '-') ~ Day14.obob.FGF1vsVehPF,
                                  TRUE ~ '-')) %>%
    mutate(FGF1_both_days_same = case_when( ((FGF1_Day5 == FGF1_Day14) & (FGF1_Day5 != '-') & (FGF1_Day14 != '-')) ~ FGF1_Day5,
                                           TRUE ~ '-')) %>%
    mutate(FGF1_both_days_diff = case_when( ((FGF1_Day5 != FGF1_Day14) & (FGF1_Day5 != '-') & (FGF1_Day14 != '-')) ~ FGF1_Day5,
                                           TRUE ~ '-')) %>%
    relocate(any_of(c("FGF1_Day5", "FGF1_Day14", "FGF1_both_days_same", "FGF1_both_days_diff")), .after=module_size)
}

In [11]:
chst = qs::qread('_targets/objects/combined_hubgenes_summary_tt_all')
chst_summary = make_combined_hubgenes_summary_tt_all_summary(chst)
chst_summary %>% head

chst_summary %>% openxlsx::write.xlsx('module_comparison_summary.xlsx')

label,module_name,module_size,FGF1_Day5,FGF1_Day14,FGF1_both_days_same,FGF1_both_days_diff,Day5.obob.FGF1vsVehPF,Day14.obob.FGF1vsVehPF,Day5vsDay14.obob.VehPF,Day5vsDay14.BL6.VehPF
<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
a1_Tany__Astrocytes,a1_Tany__Astrocytes_M01,518,UP,-,-,-,UP,-,-,-
a1_Tany__Astrocytes,a1_Tany__Astrocytes_M02,317,UP,-,-,-,UP,-,-,-
a1_Tany__Astrocytes,a1_Tany__Astrocytes_M03,306,UP,-,-,-,UP,-,-,-
a1_Tany__Astrocytes,a1_Tany__Astrocytes_M04,282,UP,-,-,-,UP,-,-,-
a1_Tany__Astrocytes,a1_Tany__Astrocytes_M05,211,UP,UP,UP,-,UP,UP,-,-
a1_Tany__Astrocytes,a1_Tany__Astrocytes_M06,148,-,-,-,-,-,-,-,-


In [12]:
chst_summary %>% print

# A tibble: 1,801 × 11
   label modul…¹ modul…² FGF1_…³ FGF1_…⁴ FGF1_…⁵ FGF1_…⁶ Day5.…⁷ Day14…⁸ Day5v…⁹
   <chr> <chr>     <int> <chr>   <chr>   <chr>   <chr>   <chr>   <chr>   <chr>  
 1 a1_T… a1_Tan…     518 UP      -       -       -       UP      -       -      
 2 a1_T… a1_Tan…     317 UP      -       -       -       UP      -       -      
 3 a1_T… a1_Tan…     306 UP      -       -       -       UP      -       -      
 4 a1_T… a1_Tan…     282 UP      -       -       -       UP      -       -      
 5 a1_T… a1_Tan…     211 UP      UP      UP      -       UP      UP      -      
 6 a1_T… a1_Tan…     148 -       -       -       -       -       -       -      
 7 a1_T… a1_Tan…     128 UP      -       -       -       UP      -       -      
 8 a1_T… a1_Tan…      95 -       -       -       -       DOWN    -       UP     
 9 a1_T… a1_Tan…      89 UP      -       -       -       UP      -       -      
10 a1_T… a1_Tan…      88 UP      UP      UP      -       UP      UP      -      
# … w

In [13]:
chst_summary %>%
group_by(label) %>%
count(FGF1_Day5) %>%
ungroup %>%
mutate(across(2, ~ifelse(. == '-', 'ns', .))) %>%
rename(direction=2) %>%
pivot_wider(names_from = direction, values_from=n)  %>%
mutate(across(c(2:4), ~replace_na(., 0))) %>%
rowwise %>%
mutate(total_modules = sum(ns, DOWN, UP)) %>%
mutate(DOWN_frac = DOWN/total_modules) %>%
mutate(UP_frac = UP/total_modules) %>%
mutate(total_frac = UP_frac + DOWN_frac) %>%
select(label, UP, DOWN, total_modules, UP_frac, DOWN_frac, total_frac) %>%
ungroup %>%
openxlsx::write.xlsx('FGF1_Day5_module_direction_summary.xlsx')


In [14]:
chst_summary %>%
group_by(label) %>%
count(FGF1_Day5) %>%
ungroup %>%
mutate(across(2, ~ifelse(. == '-', 'ns', .))) %>%
rename(direction=2) %>%
pivot_wider(names_from = direction, values_from=n)  %>%
mutate(across(c(2:4), ~replace_na(., 0))) %>%
rowwise %>%
mutate(total_modules = sum(ns, DOWN, UP)) %>%
mutate(DOWN_frac = DOWN/total_modules) %>%
mutate(UP_frac = UP/total_modules) %>%
mutate(total_frac = UP_frac + DOWN_frac) %>%
select(label, UP, DOWN, total_modules, UP_frac, DOWN_frac, total_frac) %>%
ungroup

label,UP,DOWN,total_modules,UP_frac,DOWN_frac,total_frac
<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>
a1_Tany__Astrocytes,12,3,20,0.60000000,0.150000000,0.750000000
a2_Tanycytes__b1_Tanycytes,10,2,17,0.58823529,0.117647059,0.705882353
Agrp,1,0,12,0.08333333,0.000000000,0.083333333
Astrocytes,2,1,10,0.20000000,0.100000000,0.300000000
b1_Tanycytes,0,1,5,0.00000000,0.200000000,0.200000000
b2_Tany,1,0,77,0.01298701,0.000000000,0.012987013
Epend,1,2,8,0.12500000,0.250000000,0.375000000
Ghrh,0,2,97,0.00000000,0.020618557,0.020618557
Gm8773_Tac1__Hdc,0,1,12,0.00000000,0.083333333,0.083333333


In [15]:
chst_summary %>%
group_by(label) %>%
count(FGF1_Day14) %>%
ungroup %>%
mutate(across(2, ~ifelse(. == '-', 'ns', .))) %>%
rename(direction=2) %>%
pivot_wider(names_from = direction, values_from=n)  %>%
mutate(across(c(2:4), ~replace_na(., 0))) %>%
rowwise %>%
mutate(total_modules = sum(ns, DOWN, UP)) %>%
mutate(DOWN_frac = DOWN/total_modules) %>%
mutate(UP_frac = UP/total_modules) %>%
mutate(total_frac = UP_frac + DOWN_frac) %>%
select(label, UP, DOWN, total_modules, UP_frac, DOWN_frac, total_frac) %>%
ungroup %>%
openxlsx::write.xlsx('FGF1_Day14_module_direction_summary.xlsx')


In [16]:
chst_summary %>%
group_by(label) %>%
count(FGF1_both_days_same) %>%
ungroup %>%
mutate(across(2, ~ifelse(. == '-', 'ns', .))) %>%
rename(direction=2) %>%
pivot_wider(names_from = direction, values_from=n)  %>%
mutate(across(c(2:4), ~replace_na(., 0))) %>%
rowwise %>%
mutate(total_modules = sum(ns, DOWN, UP)) %>%
mutate(DOWN_frac = DOWN/total_modules) %>%
mutate(UP_frac = UP/total_modules) %>%
mutate(total_frac = UP_frac + DOWN_frac) %>%
select(label, UP, DOWN, total_modules, UP_frac, DOWN_frac, total_frac) %>%
ungroup %>%
openxlsx::write.xlsx('FGF1_both_days_module_direction_summary.xlsx')


In [17]:
qs::qread('_targets/objects/combined_hubgenes_summary_all') %>%
openxlsx::write.xlsx('combined_hubgenes_summary_all.xlsx')

In [18]:
qs::qread('_targets/objects/combined_gost_tt_summary_all') %>%
openxlsx::write.xlsx('combined_gost_tt_summary_all.xlsx')